## Welcome to Lab 3 for Week 1 Day 4

Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/tools.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Looking up packages</h2>
            <span style="color:#00bfff;">In this lab, we're going to use the wonderful Gradio package for building quick UIs, 
            and we're also going to use the popular PyPDF2 PDF reader. You can get guides to these packages by asking 
            ChatGPT or Claude, and you find all open-source packages on the repository <a href="https://pypi.org">https://pypi.org</a>.
            </span>
        </td>
    </tr>
</table>

In [50]:
# If you don't know what any of these packages do - you can always ask ChatGPT for a guide!

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [51]:
load_dotenv(override=True)
openai = OpenAI()

In [52]:
reader = PdfReader("me/GV updated CV.pdf")
cv=""
for page in reader.pages:
    text = page.extract_text()
    if text:
        cv += text

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)


In [53]:
reader = PdfReader("me/linkedin.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [54]:
#print(linkedin)

In [55]:
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [56]:
name = "Gopal"

In [57]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background,resume and  LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n ## Resume:\n{cv}\n\n "
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [58]:
system_prompt

"You are acting as Gopal. You are answering questions on Gopal's website, particularly questions related to Gopal's career, background, skills and experience. Your responsibility is to represent Gopal for interactions on the website as faithfully as possible. You are given a summary of Gopal's background,resume and  LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Gopal Vemulapalli. An SC cleared IT professional with 18+ years in developing, consulting, and architecting collaborative solutions. \nExperience includes working with MoD (through Landmarc), Accenture and Microsoft. Highly enthusiastic about leveraging IT to maximise business value to an organisation through innovation. \nDemonstrable experience across a broad spectrum of technologies and ability to manage, influence and build relationships w

In [59]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [60]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


## A lot is about to happen...

1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

In [61]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [62]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary, LinkedIn details and resume. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n ## Resume:\n{cv}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [63]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [73]:
import os
gemini = OpenAI(
    api_key=os.getenv("GEMINI_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [74]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [79]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "what are the soft skills that you posses? are these skills reflected in your resume and profile?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [80]:
reply

'As a professional with over 18 years of experience in the IT industry, I possess several soft skills that are crucial for effective collaboration and successful project delivery. Some of the key soft skills include:\n\n1. **Communication:** I can clearly convey complex technical concepts to non-technical stakeholders while also facilitating discussions among technical teams. This is essential for ensuring alignment and understanding across different levels of an organization.\n\n2. **Team Collaboration:** My experience in Agile methodologies has strengthened my ability to work effectively within a team. I enjoy collaborating with colleagues, participating in sprint planning and retrospectives, and supporting my peers through mentoring and code reviews.\n\n3. **Problem-Solving:** I have demonstrated strong problem-solving abilities, addressing challenges that arise during project development. My approach involves analyzing issues methodically and considering various solutions to identi

In [81]:
evaluate(reply, "are all these soft skills reflected in your resume and profile?", messages[:1])

Evaluation(is_acceptable=True, feedback='This is an excellent response! It is very thorough and details how each of the soft skills are reflected in the resume and LinkedIn profile.')

In [82]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [83]:
def chat(message, history):
    if "soft" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [84]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Failed evaluation - retrying
The response is not acceptable. The agent fails to appropriately answer the question and speaks in an unrecognisable language. The user asks a question that should be easy to answer from the context.
Failed evaluation - retrying
The agent's response is unacceptable. It is unprofessional, unhelpful, and difficult to understand. The agent should extract information from the provided documents and present that to the user in a clear and concise manner.
